In [1]:
# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
import xgboost
from xgboost import XGBClassifier
from random import randint
import  random
import os
os.chdir('C:/Users/Travis/OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from cmath import nan
import urllib
import urllib.request
import re
import time


#### Get sitemap for fighters

In [2]:
url = 'https://www.tapology.com/fightcenter/fighters/sitemap.xml'
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
r = requests.get(url, headers = header)
r.text

# turn into df
df = pd.DataFrame(r.text.split('\n'))
df.columns = ['url']
df = df[df['url'].str.contains('fighter')]
df

,url
3,<loc>https://www.tapology.com/fightcenter/...
9,<loc>https://www.tapology.com/fightcenter/...
15,<loc>https://www.tapology.com/fightcenter/...
21,<loc>https://www.tapology.com/fightcenter/...
27,<loc>https://www.tapology.com/fightcenter/...
...,...
26835,<loc>https://www.tapology.com/fightcenter/...
26841,<loc>https://www.tapology.com/fightcenter/...
26847,<loc>https://www.tapology.com/fightcenter/...
26853,<loc>https://www.tapology.com/fightcenter/...


In [3]:
# fighter names, remove  numbers at end of names
df['fighter_first'] = df['url'].str.split('fighters/').str[1].replace('<loc></url>', "").str.split('-').str[0]
df['fighter_last'] = df['url'].str.split('fighters/').str[1].replace('<loc></url>', "").str.split('-').str[-2]

df

,url,fighter_first,fighter_last
3,<loc>https://www.tapology.com/fightcenter/...,dan,henderson
9,<loc>https://www.tapology.com/fightcenter/...,rich,franklin
15,<loc>https://www.tapology.com/fightcenter/...,mauricio,rua
21,<loc>https://www.tapology.com/fightcenter/...,mark,the
27,<loc>https://www.tapology.com/fightcenter/...,alan,the
...,...,...,...
26835,<loc>https://www.tapology.com/fightcenter/...,4993,jennifer
26841,<loc>https://www.tapology.com/fightcenter/...,4994,anthony
26847,<loc>https://www.tapology.com/fightcenter/...,4995,ana
26853,<loc>https://www.tapology.com/fightcenter/...,4996,tommy


In [21]:
df['sherdog_urls'] = df['url'].str.replace('<loc>', "").str.replace('<url>', '').str.strip()
df

,url,fighter_first,fighter_last,sherdog_urls
2,<url><loc>https://www.sherdog.com/fighter/Tony...,Tony,Galindo,https://www.sherdog.com/fighter/Tony-Galindo-1...
3,<url><loc>https://www.sherdog.com/fighter/Kurt...,Kurt,Rojo,https://www.sherdog.com/fighter/Kurt-Rojo-2</l...
4,<url><loc>https://www.sherdog.com/fighter/Bria...,Brian,Hawkins,https://www.sherdog.com/fighter/Brian-Hawkins-...
5,<url><loc>https://www.sherdog.com/fighter/Ken-...,Ken,Shamrock,https://www.sherdog.com/fighter/Ken-Shamrock-4...
6,<url><loc>https://www.sherdog.com/fighter/Alex...,Alexander,Otsuka,https://www.sherdog.com/fighter/Alexander-Otsu...
...,...,...,...,...
19997,<url><loc>https://www.sherdog.com/fighter/Geof...,Geoffrey,Minne,https://www.sherdog.com/fighter/Geoffrey-Minne...
19998,<url><loc>https://www.sherdog.com/fighter/Tim-...,Tim,Samelstad,https://www.sherdog.com/fighter/Tim-Samelstad-...
19999,<url><loc>https://www.sherdog.com/fighter/Nick...,Nick,Edwards,https://www.sherdog.com/fighter/Nick-Edwards-2...
20000,<url><loc>https://www.sherdog.com/fighter/Curt...,Curtis,Demarce,https://www.sherdog.com/fighter/Curtis-Demarce...


In [22]:
df= df.drop(['url'], axis = 1)
df

,fighter_first,fighter_last,sherdog_urls
2,Tony,Galindo,https://www.sherdog.com/fighter/Tony-Galindo-1...
3,Kurt,Rojo,https://www.sherdog.com/fighter/Kurt-Rojo-2</l...
4,Brian,Hawkins,https://www.sherdog.com/fighter/Brian-Hawkins-...
5,Ken,Shamrock,https://www.sherdog.com/fighter/Ken-Shamrock-4...
6,Alexander,Otsuka,https://www.sherdog.com/fighter/Alexander-Otsu...
...,...,...,...
19997,Geoffrey,Minne,https://www.sherdog.com/fighter/Geoffrey-Minne...
19998,Tim,Samelstad,https://www.sherdog.com/fighter/Tim-Samelstad-...
19999,Nick,Edwards,https://www.sherdog.com/fighter/Nick-Edwards-2...
20000,Curtis,Demarce,https://www.sherdog.com/fighter/Curtis-Demarce...


In [23]:
df.to_csv('data/final/aggregates/Sherdog_Fighter_URLs.csv', index = False)